Owner: **Sean MacBride** [@seanmacb](https://github.com/seanmacb/) <br>
Adapted from a notebook from **Chris  Suberlak** [@suberlak](https://github.com/lsst-ts/ts_aos_analysis/issues/new?body=@suberlak) <br>
Last Verified to Run: **2025-08-19** <br>
<!-- Software Versions:
  - `lsst_distrib`: **w_2025_45** -->

## Setup:

This notebook was run on https://usdf-rsp.slac.stanford.edu/ .  The only necessary step required to access `consDB` is to remove the `HTTP_PROXY` environmental variable if present, with eg. `del os.environ["http_proxy"]`.

The Engineering Facilities Database, or EFD, contains fine-grained telemetry from a variety of sensors and instruments, such as 
force balance offsets (hardpoint correction), applied forces (sum of all above + AOS closed-loop corrections). For example [this movie](https://rubin-obs.slack.com/archives/C07QM715AJY/p1718056301907499) was prepared with the EFD data by C. Lage. 


ConsDB generally contains exposure-level  aggregate information, i.e. either information already present in the header of each exposure,  or  in the "Transformed EFD" portion of the ConsDB,  the aggregate  of the time-series EFD data over the exposure time window with the ["EFD Transformation service".](https://rubin-obs.slack.com/archives/C07QJMQ7L4A/p1716317310421029?thread_ts=1716315549.132149&cid=C07QJMQ7L4A)  

In [37]:
from astropy.time import Time, TimeDelta
import pandas as pd
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
from lsst.daf import butler as dafButler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils import ConsDbClient
from lsst.summit.utils.utils import computeCcdExposureId
import os

## Resources on consDB 

* A grafana page with up-to-date [consDB status](https://grafana.slac.stanford.edu/d/z7FCA4Nnk/cloud-native-postgresql-cnpg?orgId=1&refresh=30s&var-DataSource=940RXge4k&var-vcluster=vcluster--usdf-summitdb&var-cluster=summit-db-replica&var-instances=All&var-namespace=summit-db-replica&var-resolution=5m&from=now-24h&to=now)

* Confluence page with documentation on [consDB access](https://rubinobs.atlassian.net/wiki/spaces/~ktl/pages/55377993/ConsDB+Usage) 

* [Schema browser](https://sdm-schemas.lsst.io/) 

* Tech-note on the [background for consDB](https://dmtn-227.lsst.io/) 

* [ConsDB transformed EFD topics](https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48836130/Consolidated+Database+Transformed+EFD+Topics)

## Connecting to consolidated database (consDB)

In [9]:
import os 
del os.environ["http_proxy"]

KeyError: 'http_proxy'

In [10]:
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
print(f'schemas:\n', client.schema())

schemas:
 ['latiss', 'lsstcam', 'lsstcamsim', 'lsstcomcam', 'lsstcomcamsim', 'startrackerfast', 'startrackernarrow', 'startrackerwide']


In [15]:
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
print(client.schema())  # list the instruments
print(client.schema('lsstcam'))  # list tables for an instrument

['latiss', 'lsstcam', 'lsstcamsim', 'lsstcomcam', 'lsstcomcamsim', 'startrackerfast', 'startrackernarrow', 'startrackerwide']
['cdb_lsstcam.ccdexposure_camera', 'cdb_lsstcam.ccdexposure', 'cdb_lsstcam.exposure', 'cdb_lsstcam.ccdvisit1_quicklook', 'cdb_lsstcam.visit1_quicklook', 'cdb_lsstcam.ccdexposure_flexdata', 'cdb_lsstcam.ccdexposure_flexdata_schema', 'cdb_lsstcam.ccdexposure_quicklook', 'cdb_lsstcam.exposure_flexdata', 'cdb_lsstcam.exposure_flexdata_schema', 'cdb_lsstcam.exposure_quicklook', 'cdb_lsstcam.visit1', 'cdb_lsstcam.ccdvisit1']


In [17]:
print(client.schema('lsstcam', 'cdb_lsstcam.ccdexposure_camera'))

{'ccdexposure_id': ('BIGINT', 'None'), 'temp_set': ('DOUBLE PRECISION', 'None'), 'ccd_temp': ('DOUBLE PRECISION', 'None')}


In [14]:
schema = 'lsstcam'
table = 'exposure'
print(f'columns (table={table}): {table} \n',
      list(client.schema('lsstcam', f'cdb_{schema}.{table}').keys()
          )
      )

columns (table=exposure): exposure 
 ['exposure_id', 'exposure_name', 'controller', 'day_obs', 'seq_num', 'physical_filter', 'band', 's_ra', 's_dec', 'sky_rotation', 'azimuth_start', 'azimuth_end', 'azimuth', 'altitude_start', 'altitude_end', 'altitude', 'zenith_distance_start', 'zenith_distance_end', 'zenith_distance', 'airmass', 'exp_midpt', 'exp_midpt_mjd', 'obs_start', 'obs_start_mjd', 'obs_end', 'obs_end_mjd', 'exp_time', 'shut_time', 'dark_time', 'group_id', 'cur_index', 'max_index', 'img_type', 'emulated', 'science_program', 'observation_reason', 'target_name', 'air_temp', 'pressure', 'humidity', 'wind_speed', 'wind_dir', 'dimm_seeing', 'focus_z', 'simulated', 'vignette', 'vignette_min', 's_region', 'scheduler_note', 'can_see_sky']


In [18]:
# https://rubin-obs.slack.com/archives/C07QJMQ7L4A/p1730392968242269?thread_ts=1730385376.310139&cid=C07QJMQ7L4A
# For investigating schema and table content 
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
schema = 'lsstcam'
table = 'ccdvisit1_quicklook'
print(f'schemas:\n', client.schema())  # list the instruments
print(f'tables (schema={schema}):\n', client.schema(schema))  # list tables for an instrument
print(f'columns (table={table}): {table}\n', list(client.schema('lsstcam', f'cdb_{schema}.{table}').keys()))

schemas:
 ['latiss', 'lsstcam', 'lsstcamsim', 'lsstcomcam', 'lsstcomcamsim', 'startrackerfast', 'startrackernarrow', 'startrackerwide']
tables (schema=lsstcam):
 ['cdb_lsstcam.ccdexposure_camera', 'cdb_lsstcam.ccdexposure', 'cdb_lsstcam.exposure', 'cdb_lsstcam.ccdvisit1_quicklook', 'cdb_lsstcam.visit1_quicklook', 'cdb_lsstcam.ccdexposure_flexdata', 'cdb_lsstcam.ccdexposure_flexdata_schema', 'cdb_lsstcam.ccdexposure_quicklook', 'cdb_lsstcam.exposure_flexdata', 'cdb_lsstcam.exposure_flexdata_schema', 'cdb_lsstcam.exposure_quicklook', 'cdb_lsstcam.visit1', 'cdb_lsstcam.ccdvisit1']
columns (table=ccdvisit1_quicklook): ccdvisit1_quicklook
 ['ccdvisit_id', 's_ra', 's_dec', 'zenith_distance', 'photometric_calib', 'psf_sigma', 'sky_bg', 'sky_noise', 'zero_point', 'seeing_zenith_500nm', 'pixel_scale', 'astrom_offset_mean', 'astrom_offset_std', 'eff_time', 'eff_time_psf_sigma_scale', 'eff_time_sky_bg_scale', 'eff_time_zero_point_scale', 'stats_mag_lim', 'mean_var', 'n_psf_star', 'psf_area', 'psf

In [19]:
day_obs = 20250728
query = f"""
SELECT e.band, e.exp_time, q.* 
from cdb_lsstcam.visit1_quicklook q, cdb_lsstcam.exposure e
where q.visit_id = e.exposure_id
and e.day_obs = {day_obs}
--order by e.seq_num desc
"""
df = client.query(query)
#df.columns
#df[['visit_id', 'eff_time_median', 'eff_time_sky_bg_scale_median', 'eff_time_psf_sigma_scale_median', 'eff_time_zero_point_scale_median']]
df[['visit_id', 'band', 'psf_sigma_median', 'sky_bg_median', 'zero_point_median']][:5]

visit_id,band,psf_sigma_median,sky_bg_median,zero_point_median
int64,str1,object,object,object
2025072800004,r,None,None,None
2025072800005,r,None,None,None
2025072800006,r,None,None,None
2025072800007,r,None,None,None
2025072800008,r,None,None,None


In [22]:
query = f"""
SELECT v.band, v.exp_time, q.*
from cdb_lsstcam.visit1_quicklook q, cdb_lsstcam.visit1 v
where q.visit_id = v.visit_id
and v.day_obs = {day_obs}
"""
df = client.query(query)
df[:5]

band,exp_time,visit_id,day_obs,seq_num,n_inputs,pixel_scale_min,pixel_scale_max,pixel_scale_median,astrom_offset_mean_min,astrom_offset_mean_max,astrom_offset_mean_median,astrom_offset_std_min,astrom_offset_std_max,astrom_offset_std_median,eff_time_min,eff_time_max,eff_time_median,eff_time_psf_sigma_scale_min,eff_time_psf_sigma_scale_max,eff_time_psf_sigma_scale_median,eff_time_sky_bg_scale_min,eff_time_sky_bg_scale_max,eff_time_sky_bg_scale_median,eff_time_zero_point_scale_min,eff_time_zero_point_scale_max,eff_time_zero_point_scale_median,stats_mag_lim_min,stats_mag_lim_max,stats_mag_lim_median,psf_ap_flux_delta_min,psf_ap_flux_delta_max,psf_ap_flux_delta_median,psf_ap_corr_sigma_scaled_delta_min,psf_ap_corr_sigma_scaled_delta_max,psf_ap_corr_sigma_scaled_delta_median,max_dist_to_nearest_psf_min,max_dist_to_nearest_psf_max,max_dist_to_nearest_psf_median,mean_var_min,mean_var_max,mean_var_median,n_psf_star_min,n_psf_star_max,n_psf_star_median,n_psf_star_total,psf_area_min,psf_area_max,psf_area_median,psf_ixx_min,psf_ixx_max,psf_ixx_median,psf_ixy_min,psf_ixy_max,psf_ixy_median,psf_iyy_min,psf_iyy_max,psf_iyy_median,psf_sigma_min,psf_sigma_max,psf_sigma_median,psf_star_delta_e1_median_min,psf_star_delta_e1_median_max,psf_star_delta_e1_median_median,psf_star_delta_e1_scatter_min,psf_star_delta_e1_scatter_max,psf_star_delta_e1_scatter_median,psf_star_delta_e2_median_min,psf_star_delta_e2_median_max,psf_star_delta_e2_median_median,psf_star_delta_e2_scatter_min,psf_star_delta_e2_scatter_max,psf_star_delta_e2_scatter_median,psf_star_delta_size_median_min,psf_star_delta_size_median_max,psf_star_delta_size_median_median,psf_star_delta_size_scatter_min,psf_star_delta_size_scatter_max,psf_star_delta_size_scatter_median,psf_star_scaled_delta_size_scatter_min,psf_star_scaled_delta_size_scatter_max,psf_star_scaled_delta_size_scatter_median,psf_trace_radius_delta_min,psf_trace_radius_delta_max,psf_trace_radius_delta_median,sky_bg_min,sky_bg_max,sky_bg_median,sky_noise_min,sky_noise_max,sky_noise_median,seeing_zenith_500nm_min,seeing_zenith_500nm_max,seeing_zenith_500nm_median,zero_point_min,zero_point_max,zero_point_median,low_snr_source_count_min,low_snr_source_count_max,low_snr_source_count_median,low_snr_source_count_total,high_snr_source_count_min,high_snr_source_count_max,high_snr_source_count_median,high_snr_source_count_total,z4,z5,z6,z7,z8,z9,z10,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20,z21,z22,z23,z24,z25,z26,z27,z28,ringss_seeing,aos_fwhm,donut_blur_fwhm,physical_rotator_angle
str1,float64,int64,int64,int64,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,int64,int64,int64,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
r,30.0,2025072800004,20250728,4,189,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,48.43543243408203,6353.6923828125,1528.13623046875,0,0,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4.871235370635986,102.27285766601562,50.018619537353516,None,None,None,None,No

## Resources on EFD

* [EFD documentation](https://sasquatch.lsst.io/user-guide/observatorytelemetry.html)
* [Query examples](https://github.com/lsst-sqre/system-test/tree/main/efd_examples)
* [Documentation for the python connection client](https://efd-client.lsst.io/api.html)

In [26]:
butler = dafButler.Butler('/repo/embargo_new', 
                          collections=["LSSTCam/raw/all", 
                                       "LSSTCam/calib", 
                                       # "LSSTCam/nightlyValidation"
                                                           ]
                         )
client = makeEfdClient()

Given eg. RubinTV, we can find two defocal pairs to query EFD for the time corresponding to just before the first exposure, and just after the last exposure. For instance, on  `20241030` seq_num `72` and `73`. One way to get the time spans is to query the registry, and look at `min`, `max` values of `dataId.exposure.timespan`

In [27]:
dataRefs = list(butler.registry.queryDatasets('postISRCCD', where="instrument='LSSTCam' and \
exposure.observation_type='acq' and day_obs = 20250728 and exposure.seq_num in (72,73)").expanded())

Or we can use the `butlerUtils` package to obtain the exposure record (including timespan) for the first and last seqNum:

In [29]:
expId = 2025072800099
dataId = {'exposure': expId, 'detector': 97, 'instrument': 'LSSTCam'}
expRecordBegin = butlerUtils.getExpRecordFromDataId(butler, dataId)

In [30]:
expId = 2025072800159
dataId = {'exposure': expId, 'detector': 97, 'instrument': 'LSSTCam'}
expRecordEnd = butlerUtils.getExpRecordFromDataId(butler, dataId)

In [31]:
expRecordBegin.timespan.begin

<Time object: scale='tai' format='jd' value=2460885.602868922>

In [32]:
expRecordEnd.timespan.end

<Time object: scale='tai' format='jd' value=2460885.6409620717>

In [33]:
td = expRecordEnd.timespan.end-expRecordBegin.timespan.begin
print('Time it took to take the ToO exposures:', td.sec, ' sec') 

Time it took to take the ToO exposures: 3291.2481344679995  sec


### Displaying EFD topics
A list of EFD topics is available 

In [34]:
topics = await client.get_topics()
len(topics)

3095

A huge number of topics is available, to filter, let's select those with `lsst.sal`:

In [35]:
# from https://rubin-obs.slack.com/files/U07D8TLA3C3/F07QJPNTUE7/searching_the_efd
tt = [t.split('.') for t in topics]
topic_dict = {}
for t in tt:
    i = 0
    dd = topic_dict
    while i < len(t):
        if t[i] not in dd.keys():
            dd[t[i]] = {}
        dd = dd[t[i]]
        i += 1

# useful topics are 'lsst.sal.X' 
print(list(topic_dict['lsst']['sal'].keys()))

['ATAOS', 'ATArchiver', 'ATBuilding', 'ATCamera', 'ATDome', 'ATDomeTrajectory', 'ATHeaderService', 'ATHexapod', 'ATMCS', 'ATMonochromator', 'ATOODS', 'ATPneumatics', 'ATPtg', 'ATSpectrograph', 'ATWhiteLight', 'AdamSensors', 'CBP', 'CCArchiver', 'CCCamera', 'CCHeaderService', 'CCOODS', 'DIMM', 'DREAM', 'DSM', 'Dome', 'EAS', 'EPM', 'ESS', 'Electrometer', 'Environment', 'FiberSpectrograph', 'GCHeaderService', 'GIS', 'GenericCamera', 'HVAC', 'Hexapod', 'LEDProjector', 'LaserTracker', 'LinearStage', 'MTAOS', 'MTAirCompressor', 'MTCamera', 'MTDome', 'MTDomeTrajectory', 'MTHeaderService', 'MTHexapod', 'MTM1M3', 'MTM1M3TS', 'MTM2', 'MTMount', 'MTOODS', 'MTPtg', 'MTReflector', 'MTRotator', 'MTVMS', 'NewMTMount', 'OCPS', 'PMD', 'Rotator', 'Scheduler', 'Script', 'ScriptQueue', 'Test', 'TunableLaser', 'Watcher', 'WeatherForecast', 'WeatherStation']


In [36]:
# Pick a CSC and see more about its topics
ss = 'MTCamera'
for tt in topic_dict['lsst']['sal'][ss].keys():
    topic = f'lsst.sal.{ss}.{tt}'
    print(topic)
    try:
        fields = await client.get_fields(topic)
    except: 
        fields  = ['value']
    fields = [f for f in fields if ('private' not in f) and (f != 'name') and (f != 'duration')]
    try:
        dd = await client.select_top_n(topic, fields, 1)
    except: 
        print('query failed')
    display(dd)    
	

lsst.sal.MTCamera.ackcmd


,ack,cmdtype,error,identity,origin,result,timeout
2025-08-19 01:18:54.258841+00:00,303,22,0,Script:300056,23024,Done,0


lsst.sal.MTCamera.chiller_Chiller


,cascadeSetPoint,chillerPumpLife,chillerValveAct,compressorLife,controllerLife,coolPercentage,flowRate,flowSetPoint,fluidTemperature,heatPercentage,...,pressureIn,pressureOut,pressureTank,tankLevel,tankSetPoint,tempCondenser,tempDynamicSetPt,tempSetPoint,tempStage2Evap,tempTXVBulb
2024-12-11 17:31:16.716539+00:00,12.6,200,54666,202,637,0,0,0.5,12.6,0,...,5,0.5,0,0,0,8.11,20,20,8.43,8.31


lsst.sal.MTCamera.chiller_FParam_Trending


...

,avgAirtempOut,deltaPressFilt,deltaPressTotal,deltaTempAct,fanRunTime,fanSpeed,humidity,preFiltPress,retnAirTemp,retnPress,splyAirTemp,splyPress,valvePosn
2025-08-19 16:30:15.459191+00:00,10.979878,0.124672,0.285433,4.007766,6326.841977,5280,9.580052,0.629921,8.466135,0.354331,2.041833,0.505249,77.427822


lsst.sal.MTCamera.utiltrunk_UT


,averageTemp,coolFlowRate,coolHeatXfer,coolPipeRetnTemp,coolPipeSplyTemp,coolReturnPrs,coolSupplyPrs,domeXMinusTemp,domeYMinusTemp,fanInletTemp,...,fanSpeed,midXMinusTemp,midXPlusTemp,suppXMinusTemp,suppXPlusTemp,topXMinusTemp,topXPlusTemp,valvePosn,w2Q1Temp,w4Q3Temp
2025-08-19 16:30:15.471036+00:00,5.895585,4.170604,790.514917,2.639442,1.643426,None,None,18.014763,17.939498,6.125498,...,4860,4.648253,7.920037,5.283076,6.259728,5.527239,5.9179,80.774278,6.992217,5.331909


lsst.sal.MTCamera.utiltrunk_VPC


,deltaPressFilt,deltaPressTotal,deltaTempAct,fanRunTime,fanSpeed,htrCurrent,humidity,preFiltPress,retnAirTemp,retnPress,splyAirTemp,splyAirVel,splyPress,valvePosn
2025-08-19 16:30:15.481647+00:00,0.209974,1.354987,0.474768,6513.518207,6180,0,6.135171,1.423885,2.241036,0.062336,6.660105,0.013123,1.21063,74.048556


## From here, you should be able to query consDB and/or EFD, and make the plots you need!